In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install langchain langchain-community langchain-groq -q
!pip install groq PyMuPDF faiss-cpu sentence-transformers -q

import os, fitz, faiss, numpy as np
from kaggle_secrets import UserSecretsClient
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

# Setup
os.environ["GROQ_API_KEY"] = UserSecretsClient().get_secret("GROQ_API_KEY")
llm = ChatGroq(model_name="llama-3.3-70b-versatile")

# Load PDF and extract text
def extract_text(path): return "\n".join(page.get_text() for page in fitz.open(path))
text = extract_text("/kaggle/input/ipc-pdf/IPC_186045.pdf")

# Create FAISS index
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_text(text)
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-V2")
embeddings = model.encode(chunks)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

# Search and respond
def search_faiss(query):
    q_embed = model.encode([query])
    _, idx = index.search(np.array(q_embed), k=1)
    return chunks[idx[0][0]]

prompt = PromptTemplate(
    input_variables=["ipc_section", "query"],
    template="""You are an expert in Indian law. A user asked: "{query}"
Based on the Indian Penal Code (IPC), the relevant section is:
{ipc_section}
Please provide:
- A simple explanation
- The key legal points
- Possible punishments
- A real-world example"""
)

def ipc_bot(query):
    section = search_faiss(query)
    return LLMChain(llm=llm, prompt=prompt).run(ipc_section=section, query=query)

print(ipc_bot("What is the punishment for theft under IPC?"))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/tmp/ipykernel_35/1527004487.py:48: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  return LLMChain(llm=llm, prompt=prompt).run(ipc_section=section, query=query)
/tmp/ipykernel_35/1527004487.py:48: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return LLMChain(llm=llm, prompt=prompt).run(ipc_section=section, query=query)


**Simple Explanation:**
The punishment for theft under the Indian Penal Code (IPC) depends on the circumstances and severity of the crime. Theft is considered a serious offense, and the law provides for various punishments, including imprisonment and fines.

**Key Legal Points:**

1. **Section 378 of the IPC**: This section defines theft and prescribes the punishment, which can be imprisonment for up to 3 years, or a fine, or both.
2. **Section 390 of the IPC**: This section defines robbery, which is a more serious form of theft that involves violence or intimidation. Robbery is punishable under Section 392 of the IPC.
3. **Section 392 of the IPC**: This section prescribes the punishment for robbery, which can be imprisonment for up to 10 years, or a fine, or both.
4. **Aggravated forms of theft**: If the theft is committed in certain circumstances, such as in a dwelling house, or with the use of weapons, the punishment can be more severe.

**Possible Punishments:**

1. **Imprisonment*